![title](img/header.png)

In [1]:
#!pip install mlxtend --user

In [20]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [21]:
df1 = df = pd.read_excel('Online Retail2.xlsx')

In [22]:
df.head

<bound method NDFrame.head of     InvoiceNo StockCode Description  Quantity             InvoiceDate  \
0           1    85123A        roti         6 2010-12-01 08:26:00.000   
1           1     71053       selai         6 2010-12-01 08:26:00.000   
2           1    84406B     mentega         8 2010-12-01 08:26:00.000   
3           2    84029G        roti         6 2010-12-01 08:30:00.000   
4           2    84029E     mentega         6 2010-12-02 08:29:59.995   
5           3     22752        roti         2 2010-12-01 08:35:00.000   
6           3     21730        susu         6 2010-12-02 08:35:00.000   
7           3     22633     mentega         6 2010-12-03 08:35:00.000   
8           4     22632      coklat         6 2010-12-01 08:40:00.000   
9           4     84879        roti        32 2010-12-02 08:39:59.995   
10          4     22745        susu         6 2010-12-03 08:39:59.995   
11          4     22748     mentega         6 2010-12-04 08:39:59.995   
12          5     227

In [23]:
df.dtypes

InvoiceNo               int64
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
City                   object
dtype: object

In [24]:
df.describe()

,InvoiceNo,Quantity,UnitPrice,CustomerID
count,14.000000,14.000000,14.000000,14.000000
mean,3.000000,8.071429,3.204286,33.000000
std,1.414214,7.065238,1.548899,15.556349
min,1.000000,2.000000,1.690000,11.000000
25%,2.000000,6.000000,2.100000,22.000000
50%,3.000000,6.000000,3.070000,33.000000
75%,4.000000,7.500000,3.660000,44.000000
max,5.000000,32.000000,7.650000,55.000000


In [25]:
# Clean up spaces in description and remove any rows that don't have a valid invoice
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

In [27]:
#df['InvoiceNo'] = df['InvoiceNo'].astype('str')
#df = df[~df['InvoiceNo'].str.contains('C')]

In [29]:
#df.head

In [47]:
basket = (df[df['City'] =="Malang"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [31]:
basket.head()

Description,coklat,mentega,roti,selai,susu
InvoiceNo,,,,,
1,0.0,8.0,6.0,6.0,0.0
2,0.0,6.0,6.0,0.0,0.0
5,8.0,0.0,0.0,0.0,9.0


In [32]:
# Show a subset of columns
basket.iloc[:,[0,1,2]].head()
#basket.iloc[:,[0,1,2,3,4,5,6, 7]].head()

Description,coklat,mentega,roti
InvoiceNo,,,
1,0.0,8.0,6.0
2,0.0,6.0,6.0
5,8.0,0.0,0.0


In [33]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('mentega', inplace=True, axis=1)

In [35]:
frequent_itemsets = apriori(basket_sets, min_support=0.3, use_colnames=True)

In [36]:
frequent_itemsets

,support,itemsets
0,0.333333,(coklat)
1,0.666667,(roti)
2,0.333333,(selai)
3,0.333333,(susu)
4,0.333333,"(susu, coklat)"
5,0.333333,"(selai, roti)"


In [37]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.3)

In [41]:
rules.iloc[:,[0,1,2,3,4,5,6]].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(susu),(coklat),0.333333,0.333333,0.333333,1.0,3.0
1,(coklat),(susu),0.333333,0.333333,0.333333,1.0,3.0
2,(selai),(roti),0.333333,0.666667,0.333333,1.0,1.5
3,(roti),(selai),0.666667,0.333333,0.333333,0.5,1.5


In [43]:
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.8) ];

In [44]:
rules.iloc[:,[0,1,2,3,4,5,6]].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(susu),(coklat),0.333333,0.333333,0.333333,1.0,3.0
1,(coklat),(susu),0.333333,0.333333,0.333333,1.0,3.0
2,(selai),(roti),0.333333,0.666667,0.333333,1.0,1.5
3,(roti),(selai),0.666667,0.333333,0.333333,0.5,1.5


In [48]:
basket2 = (df[df['City'] =="Jakarta"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
#basket_sets2.drop('mentega', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.3, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 1) &
        (rules2['confidence'] >= 0.8)];

rules2.iloc[:,[0,1,2,3,4,5,6]].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(coklat),(mentega),0.5,1.0,0.5,1.0,1.0
1,(mentega),(coklat),1.0,0.5,0.5,0.5,1.0
2,(roti),(coklat),1.0,0.5,0.5,0.5,1.0
3,(coklat),(roti),0.5,1.0,0.5,1.0,1.0
4,(susu),(coklat),1.0,0.5,0.5,0.5,1.0


![title](img/thumbs-up.png)